TONG HOP CODE FOR TRAINING 

In [1]:
!pip install torchsummary

In [2]:
import cv2


In [3]:
#import library
import os
import cv2
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.autograd import Variable

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from sklearn.utils import shuffle

#from torchsummary import summary
#define model for training as class
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1) 
        # (channels, features matrix dimension, kernel size, padding)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)  # Calculate input size for fully connected layer
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 30)  # Adjust output size to match your number of classes

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128 * 8 * 8)  # Flatten the output from the convolutional layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)  # Apply softmax for multi-class classification
        return x

#function for sorting the classes: 
def custom_sort(classes):
    numbers = []
    characters = []
    for item in classes:
        if item.isdigit():
            numbers.append(str(item))  # Convert to int for numerical sorting
        else:
            characters.append(item)
    numbers.sort()
    characters.sort()
    return numbers + characters
#LOAD DATA AND ITS LABEL INTO 2 TENSOR
train_path = '/kaggle/input/new-ocr-dot-matrix/trainingV2/train'
#prepare data 
classes = os.listdir(train_path)
classes = custom_sort(classes) 
#read all folder to store label into an array
print(classes)
imgs_tensor = []
labels_tensor=[]
for item in classes:
    #print(os.path.join(path, folder))
    img_folder = os.path.join(train_path, item)        
    for image in os.listdir(img_folder):
        #input image
        image_path = os.path.join(img_folder, image)
        #|img = cv2.imread(image_path,cv2.COLOR_BGR2GRAY)
        img = cv2.imread(image_path)
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_AREA)
        img = np.array(img, dtype=np.float32)
        img = img.transpose((2, 0, 1))
        img = img/255     
        #img = transforms.ToTenSor(img)  
        img = img[np.newaxis, ...]
        #img = img[np.newaxis, ...]
        img_tensor = torch.from_numpy(img)     
        imgs_tensor.append(img_tensor)

    #output label with numpy 
        label = np.array(classes.index(item))
        label = label[np.newaxis]
        label_tensor = torch.from_numpy(label)   
        labels_tensor.append(label_tensor)
    #output
        #index = np.array(classes.index(item))
        #labels_tensor[i][index]=1
X = torch.cat(imgs_tensor, dim=0)
#dim = 0, concatenate by row
#dim = 1, concatenate by column
Y = torch.cat(labels_tensor, dim=0)
X_sh, y_sh = shuffle(X, Y, random_state=42)
#load 2 tensor to dataset
train_dataset = torch.utils.data.TensorDataset(X, Y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

#Call model 
model = CNNModel()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.0001)

print("Setup done---")
print(X.shape)
print(Y.shape)

['2', '3', '4', '5', '6', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'W', 'X', 'Y', 'Z']
Setup done---
torch.Size([6588, 3, 64, 64])
torch.Size([6588])


In [4]:
#Training model on pytorch
num_epochs = 20
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0
    j=0
    for i, (data, target) in enumerate(train_loader):
        # Forward pass, track output and targets
        outputs = model(data)
        #output dimension is (minibatch_size, number of classes)
        value, predicted = torch.max(outputs, 1)
        total_correct += (predicted == target).sum().item()
        total_samples += target.size(0)

        # Backward pass and update parameters (unchanged)
        loss = criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    accuracy = 100 * total_correct / total_samples
    print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}%')
model_path = '/kaggle/working/model.pt'
torch.save(model.state_dict(), model_path)
print("Training Done------")

Epoch 1: Accuracy = 23.74%
Epoch 2: Accuracy = 46.07%
Epoch 3: Accuracy = 52.26%
Epoch 4: Accuracy = 56.06%
Epoch 5: Accuracy = 57.59%
Epoch 6: Accuracy = 65.73%
Epoch 7: Accuracy = 74.91%
Epoch 8: Accuracy = 78.40%
Epoch 9: Accuracy = 79.40%
Epoch 10: Accuracy = 84.47%
Epoch 11: Accuracy = 85.05%
Epoch 12: Accuracy = 85.05%
Epoch 13: Accuracy = 85.05%
Epoch 14: Accuracy = 86.37%
Epoch 15: Accuracy = 89.01%
Epoch 16: Accuracy = 91.62%
Epoch 17: Accuracy = 91.62%
Epoch 18: Accuracy = 91.62%
Epoch 19: Accuracy = 91.62%
Epoch 20: Accuracy = 91.62%
Training Done------
